In [1]:
# This code is an implementation of the model proposed by Foster Dayan and Morris (Hippocampus, 2000). 
# This is the DMP algorithm in which every day the location of the platform changes and every trial the 
# start location of the rat changes. We can define the number of rats (= number of independant experiment 
# to perform statistics), the number of days and the number of trials per day. 
# This particular code is the implementation of their second model in which they store an estimate of 
# the positions to perform better. 

In [2]:
using Polynomials

In [3]:
###################################################################################
###################################################################################
########################                                ###########################
########################       DEFINE CLASSES           ###########################
########################                                ###########################
########################                                ##########################
########################                                ##########################
###################################################################################
###################################################################################


In [4]:
type Trial
    Trajectory
    Latency
    SearchPreference
    ActionMap
    Valuemap
    Error
    xweight
    yweight
end

In [5]:
type Day 
    trial::Any
    Day()=new(Trial[]);
    Platform::Any
end

In [6]:
type Experiment 
    day::Any
        Experiment()=new(Day[])

    PlaceCells::Any
end

In [7]:
type Rat
    experiment::Any
    Rat()=new(Experiment[])
    parameters
    featuresexperiment
end

In [8]:
###################################################################################
###################################################################################
########################                                ###########################
########################       DEFINE FUNCTIONS         ###########################
########################                                ###########################
########################                                ##########################
########################                                ##########################
###################################################################################
###################################################################################


In [9]:
#The algorithm places n points, of which the kth point is put at distance sqrt(k-1/2) from the boundary (index begins with k=1), and with polar angle 2*pi*k/phi^2 where phi is the golden ratio. Exception: the last alpha*sqrt(n) points are placed on the outer boundary of the circle, and the polar radius of other points is scaled to account for that. This computation of the polar radius is done in the function radius.

function  radius(k,n,b) # k index of point on the boundary, n number of total points, b number of boundary points
    if k>n-b
        r = 1;            # put on the boundary
    else
        r = sqrt(k-1/2)/sqrt(n-(b+1)/2);     # computation of radius of the different points 
    end
end



radius (generic function with 1 method)

In [10]:
# sunflower seed arrangement :
function sunflower(n, R, alpha)   # n number of centers,
    # alpha is indicating how much one cares about the evenness of boundary , chose 2 to have nice trade off
    # R is the radius of the circle in cm
    r=Array{Any}( n);
    theta=Array{Any}( n);
    b = round(alpha*sqrt(n));      # number of boundary points
    phi = (sqrt(5)+1)/2;           # golden ratio
    
    for k=1:n
        r[k] = R*radius(k,n,b); # computation of the radius of each point 
        theta[k] = 2*pi*k/phi^2; # computation of the angle of each point 
        
        #plot(r*cos.(theta), r*sin.(theta), "m");
    end
    # scatter(r.*cos.(theta), r.*sin.(theta));#, marker='o', "m");
    X=r.*cos.(theta); 
    Y=r.*sin.(theta);
    return hcat(X, Y)
end

Xplacecell=sunflower(493, 100, 2)[:,1];
Yplacecell=sunflower(493, 100, 2)[:,2];

In [11]:
# Define the place activity :

# Define activity as a function of position 
###### !!!!!!! POSITIONS TO BE GIVEN IN THE SAME UNITE THAN THE SIGMA ###### !!!!!!!
function place_activity(x,y,xpc,ypc,σ) # x,y 2 scalars the position of the rat, xpc,ypc 2 vectors posiions of all place cells
    N=length(xpc); # N number of place cells 
    actplacecell=zeros(N,1); # define empty array of activity 
    
    for k=1:N # k is the k-th place cell
        actplacecell[k]=exp(-((x-xpc[k])^2+(y-ypc[k])^2)/(2σ^2));
    end
    return actplacecell
end 

place_activity (generic function with 1 method)

In [12]:
function  placecells(position,centres,width)
# PLACECELLS Calculates the activity of the place cells in the simulation.
#
#	F = PLACECELLS(POSITION,CENTRES,WIDTH) calculates the activity of the place cells
#	in the simulation. The returned vector F is of length N, where N is the number of place
#	cells, and it contains the activity of each place cell given the simulated rat's current
#	POSITION (a 2 element column vector). The activity of the place cells is modelled as a
#	rate-of-fire (i.e. a scalar value) determined by a gaussian function. The CENTRES of the
#	gaussian functions are an argument, and must be a 2 x N matrix containing each place
#	cell's preferred location in 2D space. The WIDTH of the place cell fields must
#	also be provided as a scalar value (all place cells are assumed to have the same
#	width).
#
#	The returned vector, F, must be a N element column vector.
#
#	Code for BIO/NROD08 Assignment 2, Winter 2017
#	Author: Blake Richards, blake.richards@utoronto.ca


# calculate the place cell activity
F = exp.(-sum((repmat(position,1,size(centres,2))-centres).^2,1)/(2*width^2))';
return F
end


placecells (generic function with 1 method)

In [13]:
# Calculate reward as a function of position 
function reward(x,y,xp,yp,r) # x,y position of the rat and xp,yp position of the platform, r radius of the platform
    if (x-xp)^2+(y-yp)^2<= r^2 # if the rat is in the platform
        R=1;
    else # else 
        R=0;
    end 
    
end


reward (generic function with 1 method)

In [14]:
# Function to return the cumulative sum of the terms of a vector : 
function cumul(A) # A vector 
    Acum=Array{Any}(length(A));
    for k=1:length(A)
       Acum[k]=sum(A[1:k]);
    
    end
    return Acum
end

cumul (generic function with 1 method)

In [15]:
# This function tells within wich index column is located x
function indice(Acum,x) # x number, Acum vector
    
    for i=1:length(Acum)
       if i==1
           if x<Acum[i]
                return i
            end
        else
            if Acum[i-1]<x<=Acum[i]
                return i
            end
        end
    end  
        
end

indice (generic function with 1 method)

In [20]:
###################################################################################
################## GENERAL THINGS THAT DONT CHANGE WITHIN TRIALS ##################
###################################################################################

# Creating the circle and the place cells:
center=[0,0];
R= 100; # Radius of the circle in cm
r=5;# Radius of the platform  in cm
radiussearchpref=20; # radius of the area in which we calculate searchpreference 

# Motion characteristic 
dt=0.1; # timestep in s 
speed=30; # speed of the rat in cm.s-1
# Different possible directions 
angles=[-3*pi/4, -2*pi/4, -pi/4, 0, pi/4, 2*pi/4, 3*pi/4, pi];


# Trial characteristic :
T=120; # maximal duration of a trial in seconds
DeltaT=15; # Interval between trials in seconds  

# Place cells 
N=493; # number of place cells 
Xplacecell=sunflower(N,R,2)[:,1]; # absciss place cells  
Yplacecell=sunflower(N,R,2)[:,2]; # y place cells 


# Place cell : method used by Blake richards 
# initialize the centres of the place cells by random unifrom sampling across the pool
arguments= rand(1,N)*2*pi;
radii= sqrt.(rand(1,N))*R;
centres= [cos.(arguments).*radii; sin.(arguments).*radii]; 
Xplacecell=centres[1,:];
Yplacecell=centres[2,:];

σ=0.30*100; # variability of place cell activity, in centimeters


# Action cells : 
n=9; # number of action cells 


# Potential positions of the platform : 
Xplatform=[0.3,0,-0.3,0,0.5,-0.5,0.5,-0.5].*R; # in cm
Yplatform=[0,0.3,0,-0.3,0.5,0.5,-0.5,-0.5].*R;# in cm

# Potential Starting positions of the rat :
Xstart=[0.95,0,-0.95,0].*R; # East, North, West, South
Ystart=[0,0.95,0,-0.95].*R;

# Define number of rats, number of days and numbers of trials per day
numberofdays=10;
numberofrats=5;
numberoftrials=4;


times=collect(0:dt:T+dt);

In [21]:

# Parameter that regulate the choice between former angle and new angle 
momentum=1.0;



# Learning variables : 
γ=0.98; # Discount factor.  they dont precise the value  
Z=0.1; # actor learning rate
W=0.01; # critic learning rate

# learning rate for position:
Wx=0.01; # learning rate for x coordinate 
Wy=0.01;  # learning rate for y coordinate 

# parameter for postion estimation 
λ=0.9;

In [ ]:
#########################################################################
#############          LOOP       1   EXPERIMENT FOR 1 DAY 1 RAT   ######################
#########################################################################

@time begin # get the time it takes to run it 

rats=Rat();
rats.parameters=[momentum,γ,Z,W]; # Save different parameters 
rats.featuresexperiment=[numberofrats, numberofdays, numberoftrials];

    
println("start of experiments")

for indexrat=1:numberofrats
    
currentexperiment=Experiment(); # Creating the experiment 
currentexperiment.PlaceCells=hcat(Xplacecell,Yplacecell); # Store location of place cells 

# Initialisation variables :
w=zeros(N,1); # weight for critic
z=zeros(N,n); # weight for action cells 
wx=zeros(N,1); # weights for x coordinate estimate 
wy=zeros(N,1); # weights for y coordinate estimate           
        
        ##########  ##########  ##########  ##########   ########## 
    ##########  ##########  START EXPERIMENT  ##########  ##########  
        ##########  ##########  ##########  ##########   ########## 

# currentexperiment=Experiment(); # Creating the experiment 
#currentexperiment.PlaceCells=hcat(Xplacecell,Yplacecell); # Store location of place cells 
    
    for indexday=1:numberofdays
        # Everyday the location of the platform changes
        # Chose platform :
        #indexplatform=rand(1:8); # take ith platform 
        #xp=Xplatform[indexplatform];
        #yp=Yplatform[indexplatform]; 
        xp=40;
        yp=40;
        
        currentday=Day(); # creating a day 
        currentday.Platform=hcat(xp,yp);  
        
        platform=0; # indicator for the acoordinate action. evry day we suppose that the rat does not know where is the platform 
        
            ##########  ##########  ##########  ##########  
        ##########  ##########  START DAY ##########  ##########  
            ##########  ##########  ##########  ##########  
           
        println("start of days")
        
        for indextrial=1:numberoftrials ##########  
            
            ## Chose starting position :
                    # Chose starting position :
              
            # just to try if it learns better
        
            indexstart=rand(1:4); # take indexstart-th starting position : chose randomnly between 4 possibilities 1 East 2 North 3 West 4 South
            
            positionstart=[Xstart[indexstart] Ystart[indexstart]];
 
            position=positionstart;
            
            # Initialize reward 
            re=0;
            
            # Initialise index to save the trajectory and the values 
            k=1;
            # initialise time 
            t=times[k];
            historyX=Float64[];
            historyY=Float64[];
            #valuemap=Float64[];
            error=Float64[];
            searchpref=0;
            arg=0;        
            timeout=0;        
            prevdir=[0 0];    
            ##########  ##########  ##########  ##########   ########## 
            ##########  ##########  START TRIAL ##########  ##########  
            ##########  ##########  ##########  ##########   ########## 
            Xplatformestimate=0;
            Yplatformestimate=0;

println("start of trial")
                while t<=T && re==0
                          println(k)
                        if t==T
                        println("t==T")
                            X=xp;
                            Y=yp;
                            position=[X Y];
                            println(position)
                            timeout=1; # if we have to put the rat on the platform then we dont reinforce the actor but only the critic
                            platform=1;
                            println(platform)
                        
                            Xplatformestimate=dot(wx,placecells([X,Y],centres,σ)); # we register our estimate of the position of the paltform
                            Yplatformestimate=dot(wy,placecells([X,Y],centres,σ));
                            #println("platform $(platform)", Xplatformestimate ; Yplatformestimate)
                            println(platform)
                        end
                        println(Xplatformestimate)
                        
                    # Store former position to be able to draw trajectory
                    push!(historyX,position[1]) 
                    push!(historyY,position[2])
                    
                    
                         ###  Compute reward ### 
                    re=reward(position[1],position[2],xp,yp,r); 
                    
                         # compute new activity of pace cells :
                    # actplacecell=place_activity(position[1],position[2],Xplacecell,Yplacecell,σ); # this function is wrong 
                    if !(k==1)
                        formeractplacecell=actplacecell; # need storing to compute the self motion estimate
                    end
                    
                    actplacecell=placecells([position[1],position[2]],centres,σ);
                
                    ### Compute Critic ###
                    C=dot(w,actplacecell); # current estimation of the future discounted reward 
                    
                    # estimate position 
                    Xestimate=dot(wx,actplacecell);
                    Yestimate=dot(wy,actplacecell);
                    positionestimate=[Xestimate Yestimate];

                    ####### Take decision and move to new position : ########
                    # Compute the activity of action cells 
    
                    #  Compute action cell activity    
                    actactioncell=transpose(z)*actplacecell; # careful z contains place cells in rows and action cells in column 
                        if maximum(actactioncell)>=100
                            actactioncell=100.*actactioncell./maximum(actactioncell); 
                        end
                    
                    # Compute probability distribution : 
                    Pactioncell=exp.(2.*actactioncell)./sum(exp.(2.*actactioncell)); 

                    # Compute summed probability distribution:
                    SumPactioncell=cumul(Pactioncell);
                    # Generate uniform number between 0 and 1 :
                    x=rand();

                    # now chose action: 
                    indexaction=indice(SumPactioncell,x); # Chose which action between the 8 psosibilities
                    
                    if indexaction==9 # if we chose the acoord action
                        if platform==0 # if we havent registered the platform position yet 
                            indexaction=rand(1:8)
                            argdecision=angles[indexaction]; # compute the coreesponding angle 
                            newdir=[cos(argdecision) sin(argdecision)];
                            dir=(newdir./(1.0+momentum).+momentum.*prevdir./(1.0+momentum));

                        elseif platform==1 # if we have registered the platform position
                            println("we chose the weird action")
                            dir=[Xplatformestimate Yplatformestimate].-positionestimate; # get the vector of displacement 
                            dir=dir./norm(dir); # normalise
                            println("we took direction $(dir)")
                        end
                        
                        
                    else
                        argdecision=angles[indexaction]; # compute the coreesponding angle 
                        newdir=[cos(argdecision) sin(argdecision)];
                        dir=(newdir./(1.0+momentum).+momentum.*prevdir./(1.0+momentum));

                        
                    end    
                        
                        prevdir=dir;
                        # arg=α*formerarg+β*argdecision; # to constrain the angle to prevent from sharp angles
                        # arg=argdecision; # not good because angles too sharp
                        # Store former position 
                        formerposition=position;
                        # Compute new position : 
                        position=position.+dt.*speed.*dir; 
                        
                        X=position[1];
                        Y=position[2];
                        Xf=formerposition[1];
                        Yf=formerposition[2];
                
                    # We code walls as reflectors :
                        if X^2+Y^2>R^2 # if we are out of the circle 
                            # find the position between former position and current position that is exactly on the circle :
                            # Create Polynomial with a parameter lambda that represent the absciss along the segment
                            # search the value of lambda for which we are crossing the circle    
                            polynom=Poly([Xf^2+Yf^2-R^2,2*X*Xf+2*Y*Yf-2*Xf^2-2*Yf^2,Xf^2+Yf^2+X^2+Y^2-2*X*Xf-2*Y*Yf]); # using poly creates a polynomial, coefficient are in order of increasing exposant 
                            # find the root of this polynomial that is between 0 and 1 (there is just one by I dont know which theorem)
                            λ=roots(polynom)[find(x -> 0<x <1,roots(polynom))];
                            λ=maximum(λ); # to convert from array of float to float 
                            Xlambda=λ*X+(1-λ)Xf; # position of the point that is on the circle 
                            Ylambda=λ*Y+(1-λ)Yf;
                            delta=norm([Xlambda-X,Ylambda-Y]); # distance of the point to Xlambda Ylambda
                                
                            #anglereflect=acos(dot([Xlambda, Ylambda],[Xf-Xlambda,Yf-Ylambda])/(norm([Xlambda, Ylambda])*norm([Xf-Xlambda,Yf-Ylambda]))); # compute the angle between the former position and the radius linking the point in the circle to the center 
                            #anglerotation=acos(Xlambda/norm([Xlambda, Ylambda])); # angle of rotation to calculate the new coordonnee, angle between the point in the circle and the x axis
                            # Find the intersection between the line starting from X,Y in the direction of Xlambda and Ylambda and the circle of centre Xlambda Ylambda of radius delta
                            poly2=Poly([Y^2-2*Ylambda*Y+(Ylambda^2)+X^2-2*Xlambda*X+(Xlambda^2)-delta^2, -2*Ylambda*Y/R+2*Ylambda^2/R-2*Xlambda*X/R+2*Xlambda^2/R ,Ylambda^2/R^2+Xlambda^2/R^2]);
            
                            # Problem with root is the precision : sometimes the first root given is reaaally near the first point in which case we want the second root
                            deplacement=maximum(roots(poly2)[find(x -> 0<x ,roots(poly2))]); 
                            
                                
                            # Compute new position : we just move following the inverse vector of Xlambda,Ylambda of the distance we computed
                            Xnew=X-deplacement*Xlambda/R;
                            Ynew=Y-deplacement*Ylambda/R;
                            #X=-delta*cos(anglerotation)*cos(anglereflect)-delta*sin(anglerotation)*sin(anglereflect)+delta*sin(anglerotation)*cos(anglereflect)+delta*cos(anglerotation)*sin(anglereflect)+Xlambda;   
                            #Y=-delta*sin(anglerotation)*cos(anglereflect)+delta*sin(anglerotation)*sin(anglereflect)-delta*cos(anglerotation)*cos(anglereflect)+delta*cos(anglerotation)*sin(anglereflect)+Ylambda;   
                                if Xnew^2+Ynew^2>R^2 # if we are still out of the circle 
                                    println("we are still out")
                                    break
                                end

                            X=Xnew;
                            Y=Ynew;
                            position=[X Y];    
                        end
                    
                    # If we are now at the very edge of the maze, move us in a little bit :
                        if X^2+Y^2==R^2
                            position = (position./(X^2+Y^2))*(R - 1);
                        end
                
                    # compute new activity of pace cells :
                    # actplacecell=place_activity(position[1],position[2],Xplacecell,Yplacecell,σ);
                    actplacecell=placecells([position[1],position[2]],centres,σ);

                        if re==1 # if we are on the platform 
                           ###  Compute error ###
                            Cnext=0;
                            platform=1;
                            Xplatformestimate=dot(wx,actplacecell); # we register our estimate of the position of the paltform
                            Yplatformestimate=dot(wy,actplacecell);
                                println("re=$(re)","platform$(platform)", Xplatformestimate)

                        else 
                            Cnext=dot(w,actplacecell);# new estimation of the future discounted reward 
                        end 
                    
                
                    #### Compute error  ####
                    err=re+γ*Cnext-C;
            
                    # save error
                    push!(error,err);
                
                
                    ######### Compute new weights : ########
                        if timeout==0
                            G=zeros(n,1);
                            G[indexaction]=1;
                            # weights between action cells and place cells only reinforced when the rats actually found the platform
                            # z[:,indexaction]=z[:,indexaction]+Z.*err.*actplacecell; # only the weights between place cells and the action taken are updated
                            z=z+Z.*err.*actplacecell*transpose(G);       
                        end
                    
                    # weights between critic and place cells :
                    # Save value to draw valuemap
                    # push!(valuemap,w);
                    w=w+W.*err.*actplacecell;
    
                     ####### ####### ####### Updating search preference  ####### ####### #######
                        if (X-xp)^2+(Y-yp)^2<= radiussearchpref^2          
                            searchpref=searchpref+1*dt;
                        end
                   
                    k=k+1;
                    t=times[k];
                      
                    
                    # uopdate the weight for position estimate 
                    # self motion estimate :
                    if !(k==2)
                         
                    deltax=dot(wx,actplacecell)-dot(wx,formeractplacecell);
                    deltay=dot(wy,actplacecell)-dot(wy,formeractplacecell);
                    wx=wx+Wx.*(deltax+X-Xf).*(sum([λ^(k-l).*placecells([historyX[l],historyY[l]],centres,σ) for l=1:(k-1)])+actplacecell);
                    wy=wy+Wy.*(deltay+Y-Yf).*(sum([λ^(k-l).*placecells([historyX[l],historyY[l]],centres,σ) for l=1:(k-1)])+actplacecell);
                    end
                    println(k)
                    
                ##################################################            
                end

                ########## ##########  END TRIAL ########## ##########             
            println("wearehere")
            push!(historyX,position[1]) # Store the last position visited 
            push!(historyY,position[2])
            # push!(valuemap,w)
                 println("wearehere2")       
            ############### SAVING THE THINGS IN THE DIFFERENT CLASS ################
            ## in creating a new trial type one should write Trial(Trajectory, latency, searchpreference, actionmap) # action map atm is just z, then it will be improved adding a new attribute being value map 
            println("wearehere3")
            currenttrial=Trial(hcat(historyX,historyY),t,searchpref,z,w,error,wx,wy); # Creating the current trial with all its fields
            println("wearehere4")
            push!(currentday.trial,currenttrial) # Storing it in the current day 
        
                
        ##################################################     
        end 
        ########## ##########  END DAY ########## ##########
        
        
        push!(currentexperiment.day,currentday) # Storing the current day in the current experiment 
        
            
    ##################################################     
    end 
    ########## ##########  END EXPERIMENT ########## ##########

push!(rats.experiment,currentexperiment) # Storing the current experiment in the rat's class

##################################################     
end 
########## ##########  END RATS ########## ###
    
    
end # end time 

start of experiments
start of days
start of trial
1
0
2
2
0
3
3
0
4
4
0
5
5
0
6
6
0
7
7
0
8
8
0
9
9
0
10
10
0
11
11
0
12
12
0
13
13
0
14
14
0
15
15
0
16
16
0
17
17
0
18
18
0
19
19
0
20
20
0
21
21
0
22
22
0
23
23
0
24
24
0
25
25
0
26
26
0
27
27
0
28
28
0
29
29
0
30
30
0
31
31
0
32
32
0
33
33
0
34
34
0
35
35
0
36
36
0
37
37
0
38
38
0
39
39
0
40
40
0
41
41
0
42
42
0
43
43
0
44
44
0
45
45
0
46
46
0
47
47
0
48
48
0
49
49
0
50
50
0
51
51
0
52
52
0
53
53
0
54
54
0
55
55
0
56
56
0
57
57
0
58
58
0
59
59
0
60
60
0
61
61
0
62
62
0
63
63
0
64
64
0
65
65
0
66
66
0
67
67
0
68
68
0
69
69
0
70
70
0
71
71
0
72
72
0
73
73
0
74
74
0
75
75
0
76
76
0
77
77
0
78
78
0
79
79
0
80
80
0
81
81
0
82
82
0
83
83
0
84
84
0
85
85
0
86
86
0
87
87
0
88
88
0
89
89
0
90
90
0
91
91
0
92
92
0
93
93
0
94
94
0
95
95
0
96
96
0
97
97
0
98
98
0
99
99
0
100
100
0
101
101
0
102
102
0
103
103
0
104
104
0
105
105
0
106
106
0
107
107
0
108
108
0
109
109
0
110
110
0
111
111
0
112
112
0
113
113
0
114
114
0
115
115
0
116
116
0
117
117


In [ ]:
# Save the data so we can open them again or use them to plot 


using JLD: save 

save("/Users/pmxct2/Documents/FosterDayanMorris/experimentwithposition.jld", "rats", rats)